In [ ]:
import cv2
import numpy as np

# Load the map image
try:
    map_path = "C:\\Users\\saya jahnavi\\Desktop\\download (1).png"
    map = cv2.imread(map_path, 0)
    if map is None:
        raise FileNotFoundError(f"The image file at '{map_path}' was not found.")
    
    HEIGHT, WIDTH = map.shape
    print("Image loaded successfully")
    print(f"Image dimensions: {WIDTH}x{HEIGHT}")

    rx, ry, rtheta = WIDTH / 4, HEIGHT / 4, 0
    print(f"Initial robot position: rx={rx}, ry={ry}, rtheta={rtheta}")

except FileNotFoundError as e:
    print(e)
except Exception as e:
    print(f"An unexpected error occurred: {e}")

# Constants and initialization
STEP = 5
TURN = np.radians(25)
SIGMA_STEP = 0.5
SIGMA_TURN = np.radians(5)
NUM_PARTICLES = 3000
SIGMA_SENSOR = 2
SIGMA_POS = 2

# Function to get input
def get_input():
    fwd, turn, halt = 0, 0, False
    k = cv2.waitKey(0)
    if k == 82:  # Up arrow key
        fwd = STEP
    elif k == 83:  # Right arrow key
        turn = TURN
    elif k == 81:  # Left arrow key
        turn = -TURN
    elif k == 27:  # Esc key
        halt = True
    return fwd, turn, halt

# Function to move robot
def move_robot(rx, ry, rtheta, fwd, turn):
    fwd_noisy = np.random.normal(fwd, SIGMA_STEP)
    rx += fwd_noisy * np.cos(rtheta)
    ry += fwd_noisy * np.sin(rtheta)
    turn_noisy = np.random.normal(turn, SIGMA_TURN)
    rtheta += turn_noisy
    return rx, ry, rtheta

# Initialize particles
def init_particles():
    particles = np.random.rand(NUM_PARTICLES, 3)
    particles *= np.array([WIDTH, HEIGHT, np.radians(360)])
    return particles

# Move particles
def move_particles(particles, fwd, turn):
    particles[:, 0] += fwd * np.cos(particles[:, 2])
    particles[:, 1] += fwd * np.sin(particles[:, 2])
    particles[:, 2] += turn
    particles[:, 0] = np.clip(particles[:, 0], 0.0, WIDTH-1)
    particles[:, 1] = np.clip(particles[:, 1], 0.0, HEIGHT-1)
    return particles

# Sense function
def sense(x, y, noisy=False):
    x, y = int(x), int(y)
    if noisy:
        return np.random.normal(map[y, x], SIGMA_SENSOR)
    return map[y, x]

# Compute weights
def compute_weights(particles, robot_sensor):
    errors = np.zeros(NUM_PARTICLES)
    for i in range(NUM_PARTICLES):
        elevation = sense(particles[i, 0], particles[i, 1], noisy=False)
        errors[i] = abs(robot_sensor - elevation)
    weights = np.max(errors) - errors
    weights[(particles[:, 0] == 0) | (particles[:, 0] == WIDTH-1) |
            (particles[:, 1] == 0) | (particles[:, 1] == HEIGHT-1)] = 0.0
    weights = weights ** 3
    return weights

# Resample particles
def resample_particles(particles, weights):
    probabilities = weights / np.sum(weights)
    new_indices = np.random.choice(NUM_PARTICLES, size=NUM_PARTICLES, p=probabilities)
    particles = particles[new_indices, :]
    return particles

# Add noise to particles
def add_noise_to_particles(particles):
    noise = np.concatenate([
        np.random.normal(0, SIGMA_POS, (NUM_PARTICLES, 1)),
        np.random.normal(0, SIGMA_POS, (NUM_PARTICLES, 1)),
        np.random.normal(0, SIGMA_TURN, (NUM_PARTICLES, 1))
    ], axis=1)
    particles += noise
    return particles

# Display map, robot, and particles
def display(map, rx, ry, particles):
    lmap = cv2.cvtColor(map, cv2.COLOR_GRAY2BGR)
    # Display particles
    for i in range(NUM_PARTICLES):
        cv2.circle(lmap, (int(particles[i, 0]), int(particles[i, 1])), 1, (255, 0, 0), 1)
    # Display robot
    cv2.circle(lmap, (int(rx), int(ry)), 5, (0, 255, 0), 10)
    # Display best guess
    px, py = np.mean(particles[:, 0]), np.mean(particles[:, 1])
    cv2.circle(lmap, (int(px), int(py)), 5, (0, 0, 255), 5)
    cv2.imshow('map', lmap)

# Main routine
particles = init_particles()
while True:
    display(map, rx, ry, particles)
    fwd, turn, halt = get_input()
    if halt:
        break
    rx, ry, rtheta = move_robot(rx, ry, rtheta, fwd, turn)
    particles = move_particles(particles, fwd, turn)
    if fwd != 0:
        robot_sensor = sense(rx, ry, noisy=True)
        weights = compute_weights(particles, robot_sensor)
        particles = resample_particles(particles, weights)
        particles = add_noise_to_particles(particles)
    
cv2.destroyAllWindows()


Image loaded successfully
Image dimensions: 350x310
Initial robot position: rx=87.5, ry=77.5, rtheta=0
